<a href="https://colab.research.google.com/github/Prebor97/prebor_haggai/blob/main/Neuralprophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_rainfall_data.csv to cleaned_rainfall_data.csv


In [ ]:
import pandas as pd
import numpy as np
import io
df = pd.read_csv(io.BytesIO(uploaded['cleaned_rainfall_data.csv']))

In [ ]:
corr_data = df.corr()
corr_data.abs()['pr_wtr'].sort_values(ascending=False)

pr_wtr     1.000000
tmp_y      0.723513
slp        0.715450
uwnd_x     0.676430
rhum_y     0.661081
uwnd_y     0.606514
vwnd       0.598701
rhum       0.540226
omega      0.522588
tmp        0.447026
omega_y    0.418694
omega_x    0.276757
uwnd       0.203505
rhum_x     0.067540
vwnd_y     0.061336
vwnd_x     0.028259
Name: pr_wtr, dtype: float64

In [ ]:
Main_df = df[['time','pr_wtr','tmp_y','slp','uwnd_x','rhum_y','uwnd_y','vwnd','rhum', 'omega', 'tmp', 'omega_y', 'omega_x','uwnd']]

In [ ]:
Main_df = Main_df.rename(columns = { 'time':'ds','pr_wtr': 'y'})

In [ ]:
Univariate_df = Main_df[['ds','y']]
Multivariate_df = Main_df[['ds','y','tmp_y','slp','uwnd_x','rhum_y','uwnd_y','vwnd','rhum', 'omega', 'tmp', 'omega_y', 'omega_x','uwnd']]

In [ ]:
TRAIN_SIZE = round(len(Main_df)*0.80)

In [ ]:
mutlivariateTrain_df = Multivariate_df[:TRAIN_SIZE]
mutlivariateTest_df = Multivariate_df[TRAIN_SIZE:] 

In [ ]:
test = mutlivariateTest_df.drop(columns = 'y')

In [ ]:
UnivariateTrain_df = Univariate_df[:TRAIN_SIZE]
UnivariateTest_df = Univariate_df[TRAIN_SIZE:]

In [ ]:
Utest = UnivariateTest_df.drop(columns = 'y')

In [ ]:
mutlivariateTrain_df.reset_index(inplace = True,drop = True)
mutlivariateTest_df.reset_index(inplace = True, drop = True)
test.reset_index(inplace = True, drop = True)
UnivariateTrain_df.reset_index(inplace = True, drop = True)
UnivariateTest_df.reset_index(inplace = True, drop = True)
Utest.reset_index(inplace = True, drop = True)

In [ ]:
def Evaluation_metrics(predicted, actual):
  MAE = np.mean(abs(predicted - actual))
  MSE = np.square(np.subtract(predicted, actual)).mean()
  RMSE = np.sqrt(np.mean(np.square(predicted - actual)))
  MAPE = np.mean((np.abs(np.subtract(actual, predicted)/actual)))*100
  print('The Mean Absolute Error is ', round(MAE,2))
  print('The Mean Squared Error is ', round(MSE,2))
  print('The Root Mean Squared Error is ', round(RMSE,2))
  print('The Mean Absolute Percentage Error is ', round(MAPE,2))

In [ ]:
!pip install pystan~=2.14
!pip install fbprophet 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67.3 MB 136 kB/s 
  Attempting uninstall: pystan
    Found existing installation: pystan 3.3.0
    Uninstalling pystan-3.3.0:
      Successfully uninstalled pystan-3.3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.5 MB/s 
  Created wheel for fbprophet: filename=fbprophet-0.7.1-py3-none-any.whl size=6638786 sha256=302442205aff92e928fe2f212817e7bbdcf8f28ca74f5aa757e04820315f6a44
  Stored in directory: /root/.cache/pip/wheels/cd/a1/12/db63ff624de492fe6cccf676091a0860fdde2ffde4bc3280e2
Successfully built fbprophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 1.0.7
    Uninstalling cmdstanpy-1.0.7:
      Successfully uninstalled cmdstanpy-1.0.7
ERROR: pip's dependency resolver does not currently take into account all the 

In [ ]:
from fbprophet import Prophet

#### Univariate Forecast

In [ ]:
model = Prophet()
model.fit(UnivariateTrain_df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
forecast = model.predict(Utest)

In [ ]:
forecast = forecast[['ds', 'yhat']]

In [ ]:
final_pred = pd.concat((forecast['yhat'],UnivariateTest_df), axis = 1 )
final_pred.head()

,yhat,ds,y
0,14.658550,2004-10-19,14.209991
1,14.510344,2004-10-20,14.739990
2,14.364746,2004-10-21,13.959991
3,14.205818,2004-10-22,11.459991
4,13.998301,2004-10-23,13.269989


In [ ]:
Evaluation_metrics(final_pred.yhat, final_pred.y)

The Mean Absolute Error is  4.89
The Mean Squared Error is  39.2
The Root Mean Squared Error is  6.26
The Mean Absolute Percentage Error is  42.43


#### Multivariate forecast

In [ ]:
model5 = Prophet(weekly_seasonality=False,changepoint_prior_scale=0.05, seasonality_prior_scale=10.0, seasonality_mode='additive')
for feature in mutlivariateTrain_df.columns[2:]:
  model5.add_regressor(feature)

In [ ]:
model5.fit(mutlivariateTrain_df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
forecast1 = model5.predict(test)

In [ ]:
forecast1 = forecast1[['ds', 'yhat']]

In [ ]:
final_pred1 = pd.concat((forecast1['yhat'],mutlivariateTest_df), axis = 1 )
final_pred1.head()

,yhat,ds,y,tmp_y,slp,uwnd_x,rhum_y,uwnd_y,vwnd,rhum,omega,tmp,omega_y,omega_x,uwnd
0,13.459212,2004-10-19,14.209991,294.15000,101305.0,13.030014,25.500000,3.480011,-1.389999,31.50,-0.011002,300.20000,-0.023003,0.000998,1.280014
1,16.402273,2004-10-20,14.739990,294.00000,101345.0,16.530014,40.250004,7.300003,-0.239990,34.00,-0.049002,300.18000,0.001999,0.030998,2.620010
2,14.913591,2004-10-21,13.959991,293.12000,101663.0,13.280014,20.750000,5.970001,-1.369995,37.75,0.014997,300.09998,0.054998,0.083998,0.550003
3,10.673005,2004-10-22,11.459991,292.72000,101748.0,12.550003,9.250000,2.380005,-1.469986,29.75,-0.006002,300.09998,0.051998,0.062998,3.080002
4,13.545996,2004-10-23,13.269989,292.59998,101515.0,13.950012,40.500004,1.600006,-0.839996,29.50,0.025997,300.00000,0.021997,0.054998,2.450012


In [ ]:
Evaluation_metrics(final_pred1.yhat, final_pred1.y)

The Mean Absolute Error is  2.3
The Mean Squared Error is  8.74
The Root Mean Squared Error is  2.96
The Mean Absolute Percentage Error is  21.76


#### Neural prophet

In [ ]:
! pip install neuralprophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 4.1 MB/s 
     |████████████████████████████████| 1.6 MB 47.8 MB/s 


In [ ]:
from neuralprophet import NeuralProphet

In [ ]:
# Univariate Data
univ_traindf = UnivariateTrain_df[['y', 'ds']]
univ_testdf = UnivariateTest_df[['y', 'ds']]
# Multivariate Data
multiv_traindf = mutlivariateTrain_df[['y','ds','tmp_y','slp','uwnd_x','rhum_y','uwnd_y','vwnd','rhum', 'omega', 'tmp', 'omega_y', 'omega_x','uwnd','rhum_x','vwnd_y','vwnd_x']] 
multiv_testdf = mutlivariateTest_df[['y','ds','tmp_y','slp','uwnd_x','rhum_y','uwnd_y','vwnd','rhum', 'omega', 'tmp', 'omega_y', 'omega_x','uwnd','rhum_x','vwnd_y','vwnd_x']]

#### Univariate neural prophet forecast

In [ ]:
model2 = NeuralProphet()
model2.fit(univ_traindf)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.995% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.995% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as D
INFO:NP.df_utils:Dataframe freq automatically defined as D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO:NP.config:Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO:NP.utils:Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO:NP.config:Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 85
INFO:NP.config:Auto-set epochs to 85


  0%|          | 0/157 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.76E-02, min: 2.12E+00
INFO:NP.utils_torch:lr-range-test results: steep: 8.76E-02, min: 2.12E+00


  0%|          | 0/157 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.76E-02, min: 2.77E+00
INFO:NP.utils_torch:lr-range-test results: steep: 8.76E-02, min: 2.77E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 9.15E-02
INFO:NP.forecaster:lr-range-test selected learning rate: 9.15E-02
Epoch[85/85]: 100%|██████████| 85/85 [01:08<00:00,  1.25it/s, SmoothL1Loss=0.00991, MAE=4.81, RMSE=6.08, Loss=0.00733, RegLoss=0]


,SmoothL1Loss,MAE,RMSE,Loss,RegLoss
0,0.877326,56.803551,68.079948,0.659228,0.0
1,0.319126,29.292728,35.156563,0.238752,0.0
2,0.037627,9.052289,11.237432,0.027814,0.0
3,0.010596,4.951373,6.283674,0.007766,0.0
4,0.010285,4.890505,6.197911,0.007566,0.0
...,...,...,...,...,...
80,0.009989,4.824908,6.106653,0.007390,0.0
81,0.009982,4.822051,6.104174,0.007379,0.0
82,0.009945,4.815858,6.095548,0.007353,0.0
83,0.009926,4.811558,6.089471,0.007340,0.0


In [ ]:
forecast2 = model2.predict(univ_testdf)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.981% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.981% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.981% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.981% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [ ]:
forecast2.head()

,ds,y,residual1,yhat1,trend,season_yearly,season_weekly
0,2004-10-19,14.209991,0.556098,14.766089,18.941071,-4.187902,0.012922
1,2004-10-20,14.739990,-0.144284,14.595706,18.940598,-4.388998,0.044106
2,2004-10-21,13.959991,0.477258,14.437249,18.940125,-4.586485,0.083609
3,2004-10-22,11.459991,2.777423,14.237414,18.939653,-4.780520,0.078281
4,2004-10-23,13.269989,0.745585,14.015574,18.939180,-4.971241,0.047634


In [ ]:
Evaluation_metrics(forecast2.yhat1, forecast2.y)

The Mean Absolute Error is  4.93
The Mean Squared Error is  39.69
The Root Mean Squared Error is  6.3
The Mean Absolute Percentage Error is  41.75


#### Multivariate neural prophet forecast

In [ ]:
model3 = NeuralProphet()
for feature in multiv_traindf.columns[2:]:
  model3.add_lagged_regressor(feature)

INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO:NP.forecaster:n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO:NP.forecaster:n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO:NP.forecaster:n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regr

In [ ]:
model3.fit(multiv_traindf)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.995% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.995% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as D
INFO:NP.df_utils:Dataframe freq automatically defined as D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO:NP.config:Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO:NP.utils:Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO:NP.config:Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 85
INFO:NP.config:Auto-set epochs to 85


  0%|          | 0/157 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.71E-02, min: 4.12E+00
INFO:NP.utils_torch:lr-range-test results: steep: 6.71E-02, min: 4.12E+00


  0%|          | 0/157 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.71E-02, min: 4.12E+00
INFO:NP.utils_torch:lr-range-test results: steep: 6.71E-02, min: 4.12E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 7.67E-02
INFO:NP.forecaster:lr-range-test selected learning rate: 7.67E-02
Epoch[85/85]: 100%|██████████| 85/85 [02:11<00:00,  1.55s/it, SmoothL1Loss=0.00465, MAE=3.25, RMSE=4.16, Loss=0.00348, RegLoss=0]


,SmoothL1Loss,MAE,RMSE,Loss,RegLoss
0,5.404682,256.251034,263.692873,4.065574,0.0
1,1.769878,96.511570,108.545579,1.314266,0.0
2,0.282424,26.908147,33.936716,0.205807,0.0
3,0.190162,21.585023,27.223213,0.138401,0.0
4,0.117494,16.653369,21.024068,0.085999,0.0
...,...,...,...,...,...
80,0.004718,3.270324,4.193604,0.003530,0.0
81,0.004692,3.262819,4.180919,0.003509,0.0
82,0.004674,3.254465,4.173430,0.003497,0.0
83,0.004661,3.252548,4.167487,0.003487,0.0


In [ ]:
forecast3 = model3.predict(multiv_testdf)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.981% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.981% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.981% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.981% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [ ]:
Evaluation_metrics(forecast3.yhat1, forecast3.y)

The Mean Absolute Error is  3.16
The Mean Squared Error is  16.81
The Root Mean Squared Error is  4.1
The Mean Absolute Percentage Error is  24.31


In [ ]:
metrics = {'Model' : ['Univariate fbprophet', 'Multivariate fbprophet', 'Univariate neural prophet', 'Multivariate Neural prophet'], 'MAE' : [5.11,3.51,5.0,4.06], 'MSE': [43.06,21.3,41.19,27.59], 'RMSE':[6.56,4.62,6.42,5.25],'MAPE':[38.68,23.23,38.91,29.64]}

In [ ]:
Evaluation_metrics = pd.DataFrame(metrics)
display(Evaluation_metrics.head())

,Model,MAE,MSE,RMSE,MAPE
0,Univariate fbprophet,5.11,43.06,6.56,38.68
1,Multivariate fbprophet,3.51,21.30,4.62,23.23
2,Univariate neural prophet,5.00,41.19,6.42,38.91
3,Multivariate Neural prophet,4.06,27.59,5.25,29.64


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
Evaluation_metrics.reset_index('Model', inplace = True)

In [ ]:
! pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import plotly.express as px

In [ ]:
px.bar(Evaluation_metrics, x = 'Model', y = 'MAE',title = 'Mean Absolute Error Plot')

In [ ]:
px.bar(Evaluation_metrics, x = 'Model', y = 'MSE',title = 'Mean Squared Error Plot')

In [ ]:
px.bar(Evaluation_metrics, x = 'Model', y = 'RMSE',title = 'Root Mean Squared Error Plot')

In [ ]:
px.bar(Evaluation_metrics, x = 'Model', y = 'MAPE',title = 'Mean Absolute Percentage Error Plot')

#### Of all the models, the 'MULTIVARIATE fbprophet' had the least errors. We can also infer that the addition of regressors helps increases model accuracy.
#### Generally speaking fbprophet performed better than neural prophet 

#### The arrangement and the renaming are vital parts of fbprophet and neural prophet